In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import os
import imghdr
import time
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import save_model, load_model
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.utils import shuffle

In [ ]:
# Define the brightness and contrast adjustment function
def adjust_brightness_contrast(image, brightness=40, contrast=1.0):
    # Convert to float to prevent clipping
    img = image.astype(np.float32)
    # Adjust brightness and contrast
    img = img * contrast + brightness
    # Clip to keep pixel values between 0 and 255 and convert back to uint8
    img = np.clip(img, 0, 255).astype(np.uint8)
    return img

# Ranges for brightness and contrast to test
brightness_values = range(-20, 71, 5)  # e.g., from -50 to 50 in steps of 20
contrast_values = [0.5, 0.75, 1.0, 1.25, 1.5]

In [ ]:
def is_image(file_path):
    # Returns the type of image if valid, otherwise None
    return imghdr.what(file_path) is not None

In [ ]:
# Initialize MediaPipe Hand model
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Path where images are stored for each letter in the ASL alphabet
data_dir = "../raw_data/ASL_Alphabet_Dataset/asl_alphabet_train"

landmark_data = []
labels = []

In [ ]:
# Collect landmarks for each letter
for letter in os.listdir(data_dir):
    letter_dir = os.path.join(data_dir, letter)
    letter_count = 0
    for i, img_path in enumerate(os.listdir(letter_dir)):
        if not is_image(os.path.join(letter_dir, img_path)):
            continue
        img = cv2.imread(os.path.join(letter_dir, img_path))
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = hands.process(img_rgb)
        if results.multi_hand_landmarks:
            letter_count += 1
            landmarks = []
            for lm in results.multi_hand_landmarks[0].landmark:
                landmarks.extend([lm.x, lm.y, lm.z])  # Flattened landmark vector
            landmark_data.append(landmarks)
            labels.append(letter)  # Store the label (e.g., "A", "B", etc.)
    print(letter, letter_count)

In [ ]:
# Collect landmarks for each letter with improved brighness and contrast as an alternative
for letter in os.listdir(data_dir):
    letter_dir = os.path.join(data_dir, letter)
    letter_count = 0
    for i, img_path in enumerate(os.listdir(letter_dir)):
        if not is_image(os.path.join(letter_dir, img_path)):
            continue
        img = cv2.imread(os.path.join(letter_dir, img_path))
        landmarks_found = False
        for brightness in brightness_values:
            for contrast in contrast_values:
                # Adjust brightness and contrast
                adjusted_image = adjust_brightness_contrast(img, brightness, contrast)
                # Run MediaPipe hand detection
                results = hands.process(cv2.cvtColor(adjusted_image, cv2.COLOR_BGR2RGB))
                # Check for hand landmarks and store them
                if results.multi_hand_landmarks:
                    landmarks = []
                    landmarks_found = True
                    letter_count += 1
                    for lm in results.multi_hand_landmarks[0].landmark:
                        landmarks.extend([lm.x, lm.y, lm.z])  # Flattened landmark vector
                    landmark_data.append(landmarks)
                    labels.append(letter)  # Store the label (e.g., "A", "B", etc.)
                    break
            if landmarks_found:
                #print(f'Brightness: {brightness}, Contrast: {contrast}')
                break
    print(letter, letter_count)

In [ ]:
# Convert to arrays for model input
landmark_data = np.array(landmark_data)
labels = np.array(labels)

# Save arrays to .npy files
np.save("landmark_data_large.npy", landmark_data)
np.save("labels_large.npy", labels)

# Encode labels as integers and convert to categorical
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded)

np.save('label_classes.npy', label_encoder.classes_)

In [ ]:
labels = np.load("../models/production_model/labels_large.npy")
landmark_data = np.load("../models/production_model/landmark_data_large.npy")

# Normalize landmarks relative to the wrist (landmark 0) for each frame
normalized_landmark_data = []
for frame in landmark_data:
    # Extract wrist coordinates
    wrist_x, wrist_y, wrist_z = frame[0], frame[1], frame[2]

    # Normalize each landmark in the frame relative to the wrist
    normalized_frame = []
    for i in range(0, len(frame), 3):  # Iterate over (x, y, z) coordinates
        normalized_x = frame[i] - wrist_x
        normalized_y = frame[i + 1] - wrist_y
        normalized_z = frame[i + 2] - wrist_z
        normalized_frame.extend([normalized_x, normalized_y, normalized_z])

    normalized_landmark_data.append(normalized_frame)

# Convert to numpy array
normalized_landmark_data = np.array(normalized_landmark_data)

# Encode labels as integers and convert to categorical
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded)

In [ ]:
# Label encoding for letters
y = np.array(labels).tolist()  # Convert numpy array to list of strings
y = ["Unknown" if (not isinstance(label, str) or len(label) == 0 or pd.isna(label)) else str(label) for label in y]
label_encoder = tf.keras.preprocessing.text.Tokenizer()
label_encoder.fit_on_texts(y)
y = label_encoder.texts_to_sequences(y)
y = [label[0] for label in y]  # Flatten the list of lists
y = tf.keras.utils.to_categorical(y, num_classes=len(label_encoder.word_index) + 1)
num_classes = len(label_encoder.word_index) + 1

In [ ]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(normalized_landmark_data, y, test_size=0.2, random_state=42)

In [ ]:
# Expanding the dimensions of X_train and X_test
X_train_expanded = np.expand_dims(X_train, axis=-1)
X_test_expanded = np.expand_dims(X_test, axis=-1)

In [ ]:
# Shuffle training data and labels in unison
X_train, y_train = shuffle(X_train, y_train, random_state=42)

# Optionally shuffle test data too
X_test, y_test = shuffle(X_test, y_test, random_state=42)

In [ ]:
# Build 1D CNN model
model = Sequential([
    Conv1D(64, 3, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(2),
    Conv1D(128, 3, activation='relu'),
    MaxPooling1D(2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

In [ ]:
# Build and compile the model
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

# Step 4: Train the Model
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model.fit(X_train, y_train, epochs=1000, validation_split=0.3, batch_size=16, callbacks=[early_stopping])

# Save the trained model
model.save("asl_sign_language_model.keras")

In [ ]:
# Load the trained model
#model = tf.keras.models.load_model("asl_sign_language_model.h5")
model = tf.keras.models.load_model("../models/production_model/asl_sign_language_model.keras")

In [ ]:
# Encode labels as integers and convert to categorical
labels = np.load("labels.npy")
landmark_data = np.load("landmark_data.npy")

label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded)

In [ ]:
# Prediction on webcam feed
# Initialize MediaPipe Hands and drawing utilities
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1)
mp_drawing = mp.solutions.drawing_utils

scaling_factor = np.max(landmark_data)

# Initialize webcam
cap = cv2.VideoCapture(0)

sequence = []
sequence_length = 1  # Set sequence length to 10 frames for rolling window approach - not working yet

while cap.isOpened():
    ret, frame = cap.read()
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        landmarks = []
        #for lm in results.multi_hand_landmarks[0].landmark:
        #    landmarks.extend([lm.x, lm.y, lm.z])
        # Retrieve wrist coordinates (landmark 0)
        wrist = results.multi_hand_landmarks[0].landmark[0]

        for lm in results.multi_hand_landmarks[0].landmark:
            # Normalize each landmark relative to the wrist
            normalized_x = lm.x - wrist.x
            normalized_y = lm.y - wrist.y
            normalized_z = lm.z - wrist.z
            landmarks.extend([normalized_x, normalized_y, normalized_z])

        # Draw hand landmarks on the frame
        mp_drawing.draw_landmarks(
            frame,
            results.multi_hand_landmarks[0],
            mp_hands.HAND_CONNECTIONS
        )

        # Append new frame landmarks to sequence
        sequence.append(landmarks)
        if len(sequence) > sequence_length:
            sequence.pop(0)

        # Predict only if the sequence is full
        if len(sequence) == sequence_length:
            sequence_input = np.array(sequence).flatten()[np.newaxis, ..., np.newaxis]
            #sequence_input = sequence_input/scaling_factor
            prediction = model.predict(sequence_input)
            predicted_label_index = np.argmax(prediction)
            predicted_label = label_encoder.inverse_transform([predicted_label_index])
            confidence = prediction[0][predicted_label_index] * 100  # Get confidence percentage

            # Display prediction and confidence
            cv2.putText(frame, f"Predicted: {predicted_label[0]} ({confidence:.2f}%)",
                        (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)  # Black text color

    # Display the frame
    cv2.imshow("ASL Sign Language Detection", frame)

    # Press 'C' to terminate the loop
    if cv2.waitKey(1) & 0xFF == ord("c"):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
# Prediction on images from directory
# Load the trained model
model = tf.keras.models.load_model("asl_sign_language_model_large.keras")

labels = np.load("labels.npy")
#landmark_data = np.load("landmark_data.npy")

label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded)

# Initialize MediaPipe Hands and drawing utilities
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1)
mp_drawing = mp.solutions.drawing_utils

# Root directory with test images in subdirectories
root_test_dir = "../raw_data/test_set_pics"

# Initialize a list to store processed images
processed_images = []

# Iterate through each subdirectory in the root directory
for subdir_name in os.listdir(root_test_dir):
    subdir_path = os.path.join(root_test_dir, subdir_name)

    # Ensure it is a directory
    if os.path.isdir(subdir_path):
        actual_label = subdir_name  # Use the subdirectory name as the actual label

        # Iterate through each image in the subdirectory
        for img_name in os.listdir(subdir_path):
            img_path = os.path.join(subdir_path, img_name)
            print(img_path)
            img = cv2.imread(img_path)
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            results = hands.process(img_rgb)

            if results.multi_hand_landmarks:
                # Prepare landmarks for model prediction
                landmarks = []
                wrist = results.multi_hand_landmarks[0].landmark[0]
                for lm in results.multi_hand_landmarks[0].landmark:
                    # Normalize landmarks relative to the wrist
                    normalized_x = lm.x - wrist.x
                    normalized_y = lm.y - wrist.y
                    normalized_z = lm.z - wrist.z
                    landmarks.extend([normalized_x, normalized_y, normalized_z])

                # Prepare the input for the model
                sequence_input = np.array(landmarks)[np.newaxis, ..., np.newaxis]

                # Make prediction
                prediction = model.predict(sequence_input)
                predicted_label_index = np.argmax(prediction)
                predicted_label = label_encoder.inverse_transform([predicted_label_index])
                confidence = prediction[0][predicted_label_index] * 100  # Confidence percentage

                # Draw landmarks on the image
                mp_drawing.draw_landmarks(
                    img,
                    results.multi_hand_landmarks[0],
                    mp_hands.HAND_CONNECTIONS
                )

                # Add actual label, predicted label, and confidence to the image
                cv2.putText(img, f"Actual: {actual_label}", (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 2.0, (0, 255, 0), 6)  # Green text color
                cv2.putText(img, f"Predicted: {predicted_label[0]} ({confidence:.2f}%)", (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 2.0, (0, 0, 255), 6)  # Red text color


            # Append processed image to the list
            processed_images.append(img)

# Display each processed image and check for "ESC" key press
for processed_img in processed_images:
    cv2.imshow("Processed Test Image", processed_img)
    key = cv2.waitKey(0)
    if key == 27:  # ESC key
        break

cv2.destroyAllWindows()
